### Selbstgemachte Klassen

*** 
Beispiel einer Klasse und Instanzen, die wir mittels Dictionaries emulieren wollen
***

In [14]:
class A:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    def birthday(self):
        self.age = self.age + 1
    
# Instanzen von A    
alice = A('Alice', 30)    
bob   = A('Bob', 20)    


alice.birthday()
bob.birthday()
bob.birthday()

print(alice.name, alice.age)
print(bob.name, bob.age)

Alice 31
Bob 22


### Wir wollen obigen Code wie folgt &uuml;bersetzen 
```python
class A:                  -> class_('A', init, attributs, methods)       
    attribute1 = 42
    def __init__(...):
        pass
     
    def method(self,...):
        ...
    
A(*x)                     -> instance(A, *x)
foo.x = y                 -> set_(foo, 'x', y)
foo.x                     -> get(foo, 'x')
foo.f(...)                -> get(foo, 'f')(...) 
```

***
Funktionsnamen anzeigen
***

In [21]:
def foo():
    pass
foo.__name__

'foo'

***
`class_` erstellen einen Klassendictionary, `instance` einen Instanzdictionary, und
`set_` und `get` erledigen das Schreiben und Lesen von Attributen und Methoden.
***

In [16]:
def class_(name, init=lambda x:x, attributes = {}, methods=[]):

    cls_ = {'classname': name,
            '__init__' : init,
           }
    
    # add attributes
    for k,v in attributes.items():
        cls_[k] = v
        
    # add methods    
    for m in methods:
        cls_[m.__name__] = m
        
    return cls_

def instance(cls_, *args):
    obj = {'instance_of': cls_}
    return cls_['__init__'](obj, *args)

def set_(d, k ,v):
    d[k] = v

def get(d, k):
    # key found, return value
    if k in d:
        return d[k]
    
    # key not found, look in class
    if 'instance_of' in d:
        cls_ = d['instance_of']
        if k in cls_:
            v = cls_[k]
            if not callable(v): 
                return v
            # falls v Funktion, binde die Instance d ans erste Argument
            return lambda *x:v(d, *x)    
    
    # else, raise an Exception
    if 'classname' in d:
        cls_ = d['classname']
    raise Exception('Class {} has no attribute {}!'.format(cls_, k))      

***
Zeige Informationen zu einem Klassen- oder Instanzdictionary
***

In [17]:
def info(d):
    for k,v in d.items():
        if k.startswith('_'):
            continue
            
        if k == 'instance_of':
            v = v['classname']
        elif type(v) not in [bool, None, int, float, str]:
            v = type(v).__name__
        
        print('{}: {}'.format(k.ljust(12), v)) 
    print()    

***
Obige Klasse `A` und ihre  Instanzen `alice` und `bob` mittels Dictionaries emulieren
***

In [18]:
def init(this, name, age):
    set_(this, 'name', name)
    set_(this, 'age', age)
    return this

def birthday(this):
    set_(this, 'age', get(this, 'age') + 1)
    
B = class_('B', init, methods = [birthday])    
alice = instance(B, 'Alice', 30)
bob = instance(B, 'Bob', 20)

for c in [B, alice, bob]: info(c)

classname   : B
birthday    : function

instance_of : B
name        : Alice
age         : 30

instance_of : B
name        : Bob
age         : 20



In [19]:
get(alice, 'birthday')()
get(bob, 'birthday')()
get(bob, 'birthday')()

print(get(alice, 'name'), get(alice, 'age'))
print(get(bob, 'name'), get(bob, 'age'))

Alice 31
Bob 22
